Pull trails from the database at hikingproject.com using a grid search over the rough Western Washington area.

In [25]:
# import required libraries
import config
from requests import get
from time import sleep
import json
import pandas as pd
import numpy as np

In [26]:
## variables for Hiking Project API call
# center of Seattle
latitude = 47.6062
longitude = -122.3321

# maximum distance in miles from which to pull trails
radius = 20

# maximum number of trails to return
max_trails = 500

# minimum trail length in miles
min_length = 1

Latitude and longitude don't scale directly to miles, but I believe that I have divided the ranges into small enough circles for the radius of 20 miles to cover the area.

In [27]:
## define a region to search over in terms of latitude and longitude
# longitude and latitude ranges
latitude = np.linspace(46, 49, 25)
longitude = np.linspace(-124, -120, 35)

# expand to a grid of tuples
coordinates = np.array([(lat, lon) for lat in latitude for lon in longitude])

In [31]:
## make the API calls
# declare an empty dataframe to hold the results
trails_df = pd.DataFrame()

# loop through the coordinates grid
for i in range(0, coordinates.shape[0], 1):
    # build the URL
    url = "https://www.hikingproject.com/data/get-trails?key=" + config.hiking_project_api_key + 
    "&lat=" + str(coordinates[i][0]) + "&lon="+ str(coordinates[i][1]) + "&maxDistance=" + str(radius) + 
    "&maxResults=" + str(max_trails) + "&minLength=" + str(min_length)

    # pause for a second
    sleep(1)
    
    # request data from the API
    trails = get(url)

    # build a dataframe from the JSON output
    trails_dict = trails.json()
    intermediate_df = pd.DataFrame(trails_dict["trails"])
    
    # append the intermediate dataframe to the existing results
    trails_df = pd.concat([trails_df, intermediate_df], ignore_index = "true")
    
# drop duplicates from the dataframe
trails_df.drop_duplicates(inplace = True)

In [32]:
# export the dataset as a csv
trails_df.to_csv("NWtrails.csv")